Los datos están en un archivo .h5ad. Además tenemos dos, el de facs (citometría de flujo?) y droplet. 

In [1]:
import os
import scanpy as sc
import anndata
from urllib.request import urlretrieve
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
skin_facs = anndata.read_h5ad("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\Skin_facs.h5ad")

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\anndata\compat\__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\anndata\compat\__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [3]:
skin_facs

AnnData object with n_obs × n_vars = 3468 × 22899
    obs: 'FACS.selection', 'age', 'batch', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'cellid', 'free_annotation', 'method', 'mouse.id', 'plate', 'sex', 'subtissue', 'tissue', 'well', 'n_genes', 'n_counts', 'louvain'
    var: 'n_cells', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'louvain', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'X_tsne'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

Es un anndata con los atributos que vemos arriba, donde las observaciones son las células. Sacamos la matriz de conteos. Por otro lado, la etiqueta va a ser si son células adultas (24m = 24 meses) o jóvenes (3m = 3 meses).

In [4]:
skin_facs.obs.head()

,FACS.selection,age,batch,cell,cell_ontology_class,cell_ontology_id,cellid,free_annotation,method,mouse.id,plate,sex,subtissue,tissue,well,n_genes,n_counts,louvain
index,,,,,,,,,,,,,,,,,,
A10_B001185_S46_L001.mus-2-0,Multiple,24m,0,nan,keratinocyte stem cell,CL:0002337,A10_B001185,NA,facs,24_59_M,B001185,male,nan,Skin,A10,1734,986179.0,3
A10_B002899_S262_L001.mus-2-0,Multiple,24m,0,nan,epidermal cell,CL:0000362,A10_B002899,NA,facs,24_59_M,B002899,male,Anagen,Skin,A10,2983,1833213.0,4
A11_B001185_S47_L001.mus-2-0,Multiple,24m,0,nan,keratinocyte stem cell,CL:0002337,A11_B001185,NA,facs,24_59_M,B001185,male,nan,Skin,A11,1638,747536.0,3
A11_B002899_S263_L001.mus-2-0,Multiple,24m,0,nan,keratinocyte stem cell,CL:0002337,A11_B002899,NA,facs,24_59_M,B002899,male,Anagen,Skin,A11,2306,1310311.0,1
A12_B001185_S48_L001.mus-2-0,Multiple,24m,0,nan,basal cell of epidermis,CL:0002187,A12_B001185,NA,facs,24_59_M,B001185,male,nan,Skin,A12,995,5658.0,6


In [5]:
columns_obs = skin_facs.obs.columns
data_obs = {}
for variable in columns_obs: 
  description = skin_facs.obs[variable].unique()
  data_obs[variable]= description
data_obs

{'FACS.selection': ['Multiple', 'nan']
 Categories (2, object): ['Multiple', 'nan'],
 'age': ['24m', '3m']
 Categories (2, object): ['3m', '24m'],
 'batch': ['0', '1']
 Categories (2, object): ['0', '1'],
 'cell': ['nan', 'A1.B000126.3_39_F.1.1', 'A1.B003283.3_38_F.1.1', 'A1.MAA000427.3_11_M.1.1', 'A1.MAA000435.3_10_M.1.1', ..., 'P9.MAA000549.3_8_M.1.1', 'P9.MAA000597.3_9_M.1.1', 'P9.MAA000614.3_10_M.1.1', 'P9.MAA000927.3_9_M.1.1', 'P9.MAA000938.3_8_M.1.1']
 Length: 2347
 Categories (2347, object): ['A1.B000126.3_39_F.1.1', 'A1.B003283.3_38_F.1.1', 'A1.MAA000427.3_11_M.1.1', 'A1.MAA000435.3_10_M.1.1', ..., 'P22.MAA000614.3_10_M.1.1', 'P22.MAA000927.3_9_M.1.1', 'P22.MAA000938.3_8_M.1.1', 'nan'],
 'cell_ontology_class': ['keratinocyte stem cell', 'epidermal cell', 'basal cell of epidermis']
 Categories (3, object): ['basal cell of epidermis', 'epidermal cell', 'keratinocyte stem cell'],
 'cell_ontology_id': ['CL:0002337', 'CL:0000362', 'CL:0002187']
 Categories (3, object): ['CL:0000362'

Sacamos Y. Si son jóvenes será un 0 en la etiqueta, mientras que si son adultas se codifica con un 1. 

In [6]:
import numpy as np
Y_age = skin_facs.obs["age"]
#print(Y_age)

y = []

for i in range(len(Y_age)):

  if Y_age[i]=="24m":
    #np.append(y,[1])
    y.append(1)

  else:
    #np.append(y,[-1])
    y.append(0)

Y = np.array(y)

In [9]:
Y

array([1, 1, 1, ..., 0, 0, 0])

In [10]:
np.savetxt("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\Y_skin.txt",Y,delimiter = ",")

Los nombres de los genes (genes_id) son:

In [13]:
skin_facs.n_vars

22899

In [17]:
names = skin_facs.var_names

In [19]:
print(names)

Index(['0610005C13Rik', '0610007C21Rik', '0610007L01Rik', '0610007N19Rik',
       '0610007P08Rik', '0610007P14Rik', '0610007P22Rik', '0610008F07Rik',
       '0610009B14Rik', '0610009B22Rik',
       ...
       'Zxdb', 'Zxdc', 'Zyg11a', 'Zyg11b', 'Zyx', 'Zzef1', 'Zzz3', 'a',
       'l7Rn6', 'zsGreen_transgene'],
      dtype='object', name='index', length=22899)


In [20]:
type(names)

pandas.core.indexes.base.Index

In [22]:
#np.savetxt("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\genes_id.txt",names,delimiter = ",")
pd.Series(names,index=names).to_csv("C:\matlab scripts\Tabula_Muris_Aging_Piel_Raton\genes_id_skin.csv", header=False, index=False)

Ahora sacamos la matriz de conteos. 

In [ ]:
X = skin_facs.var["n_cells"].toarray().astype(np.uint8)
X.shape

In [23]:
columns_var = skin_facs.var.columns
data_var = {}
for variable in columns_var: 
  description = skin_facs.var[variable].unique()
  data_var[variable]= description
data_var

{'n_cells': array([ 2639, 43014, 25249, ..., 18827, 16584, 21272], dtype=int64),
 'means': array([6.4001128e-02, 1.0072623e+00, 3.9597169e-01, ..., 1.5586918e-03,
        3.6520475e-01, 1.6489928e-05], dtype=float32),
 'dispersions': array([2.9700527, 2.2391338, 1.8996348, ..., 2.0661294, 2.0698838,
        0.0444914], dtype=float32),
 'dispersions_norm': array([ 0.92446554, -0.4029399 , -0.5502703 , ...,  0.00385017,
         0.00767388, -2.05512   ], dtype=float32),
 'highly_variable': array([ True, False])}